In [ ]:
!pip install datasets
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext, datasets, math
from tqdm import tqdm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dataset = datasets.load_dataset("codeparrot/github-jupyter-code-to-text")
print(dataset)

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'license', 'content'],
        num_rows: 47452
    })
    test: Dataset({
        features: ['repo_name', 'path', 'license', 'content'],
        num_rows: 11864
    })
})


In [ ]:
print(dataset['train'][333].keys())

dict_keys(['repo_name', 'path', 'license', 'content'])


In [ ]:
total_train_dataset =[]
total_test_dataset = []
for text in dataset['train']['content']:
  for split in text.split('\n'):
    if split != "":
      total_train_dataset.append(split)

for text in dataset['test']['content']:
  for split in text.split('\n'):
    if split != "":
      total_test_dataset.append(split)

print(len(total_train_dataset),len(total_test_dataset))

11367363 2875424


In [ ]:
from torchtext.data.utils import get_tokenizer
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
train_total = 300000
test_total = 3000
# tokenized_dataset_train = yield_tokens(train_set[:2*train_total])
# tokenized_dataset_test = yield_tokens(test_set[:test_total])
tokenized_dataset_train = yield_tokens(total_train_dataset[:2*train_total])
tokenized_dataset_test = yield_tokens(total_test_dataset[:test_total])

In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding
# !python3 -m spacy download en_core_web_md

2023-03-11 07:34:40.049516: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-11 07:34:40.049635: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-11 07:34:40.049655: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 20.2 MB/s

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
import re
import spacy
#spacy.load("en_core_web_s")
# import en_core_web_md
# nlp = en_core_web_md.load()
nlp = spacy.load('en_core_web_md')

def preprocessing(sentence):
    
    # Clear the html tag by using regular expression.
    sentence = re.sub("<[^>]*>", "", sentence)
    sentence = re.sub("[^\x00-\x7F]+", "", sentence) #extract non-english out
    #It matches any character which is not contained in the ASCII character set (0-127, i.e. 0x0 to 0x7F)
    stopwords = list(STOP_WORDS)
    doc = nlp(sentence)
    cleaned_tokens = []
    
    for token in doc:
        if token.text not in stopwords and token.pos_ != 'PUNCT' and token.pos_ != 'SPACE' and \
            token.pos_ != 'SYM' and token.pos_!= 'X':
                cleaned_tokens.append(token.lemma_.lower().strip())
                
    return " ".join(cleaned_tokens)

In [ ]:
from torchtext.vocab import build_vocab_from_iterator
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(preprocessing(text))

vocab = build_vocab_from_iterator(yield_tokens(total_train_dataset[:test_total]), min_freq=5) 
vocab.insert_token('<unk>', 0)           
vocab.insert_token('<eos>', 3)            
vocab.insert_token('<sos>', 2)            
vocab.insert_token('<pad>', 1)               
vocab.set_default_index(vocab['<unk>'])   
print('Vocab Size',len(vocab))                         
print(vocab.get_itos()[:10]) 

Vocab Size 385
['<unk>', '<pad>', '=', '<sos>', 'explanation', '<eos>', '#', 'end', 'import', "'"]


In [ ]:
def get_data(dataset, vocab, batch_size):
    data = []                                                   
    for example in dataset:       
        #appends eos so we know it ends....so model learn how to end...                             
        tokens = example.append('') #end of sentence
        #numericalize          
        tokens = [vocab[token] for token in example] 
        data.extend(tokens)                                    
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size 
    data = data[:num_batches * batch_size]                       
    data = data.view(batch_size, num_batches)        
    return data

In [ ]:
batch_size = 12
train_data = get_data(tokenized_dataset_train, vocab, batch_size)
valid_data = get_data(tokenized_dataset_test, vocab, batch_size)

In [ ]:
unk_idx, pad_idx, sos_idx, eos_idx = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads  #make sure it's divisible....
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        self.fc   = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.scale   = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, q, k, v, mask = None):
        b = q.shape[0]
        
        Q = self.fc_q(q)
        K = self.fc_k(k)
        V = self.fc_v(v)
        #Q, K, V = [b, l, h]
        
        #reshape them into head_dim
        #reshape them to [b, n_heads, l, head_dim]
        Q = Q.view(b, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(b, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(b, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        #Q, K, V = [b, n_heads, l, head_dim]
        
        #e = QK/sqrt(dk)
        e = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        #e: [b, n_heads, ql, kl]
        
        if mask is not None:
            e = e.masked_fill(mask == 0, -1e10)
            
        a = torch.softmax(e, dim=-1)
        
        #eV
        x = torch.matmul(self.dropout(a), V)
        #x: [b, n_heads, ql, head_dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        #x: [b, ql, n_heads, head_dim]
        
        #concat them together
        x = x.view(b, -1, self.hid_dim)
        #x: [b, ql, h]
        
        x = self.fc(x)
        #x = [b, ql, h]
        
        return x, a

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        self.fc1 = nn.Linear(hid_dim, pf_dim)
        self.fc2 = nn.Linear(pf_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.fc2(self.dropout(torch.relu(self.fc1(x))))

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()
        
        self.norm_att_norm = nn.LayerNorm(hid_dim) #second green box
        #self.enc__att_norm_ff  =  nn.LayerNorm(hid_dim) #third green box
        self.full_att_norm = nn.LayerNorm(hid_dim)
        #self.norm_maskedatt =  nn.LayerNorm(hid_dim) #first green box

        #self.multi_masked  =  MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.self_attention   =  MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)

        self.ff      =  PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout =  nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, attention = self.self_attention(enc_src, enc_src, trg, src_mask)
        
        #dropout, residual connection and layer norm #-----> first layer
        trg = self.norm_att_norm(trg + self.dropout(_trg))
            
        
        
        #positionwise feedforward  #-----> Passing into the  second layer without mask attention
        _trg = self.ff(trg) 
        
        #dropout, residual and layer norm # ------> passsing into third layer
        trg = self.full_att_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self, output_dim, hid_dim, n_layers, n_heads,
                 pf_dim, dropout, device, max_length = 100):
        super().__init__()
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pad_idx = pad_idx
        self.scale   = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        self.dropout = nn.Dropout(dropout)
        self.layers  = nn.ModuleList(
                     [
                        DecoderLayer(hid_dim, n_heads, pf_dim, dropout, device)
                        for _ in range(n_layers)
                     ]
                     )
        self.fc      = nn.Linear(hid_dim, output_dim)
        self.device  = device

    def forward(self, trg,src, enc_src=[]):
        #trg = [b, tl]
        #enc_src = hidden states from encoder = [b, sl, h]
        #trg_mask = [b, 1, tl, tl]
        #src_mask = [b, 1, 1, sl]
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        src_mask = (src != self.pad_idx).unsqueeze(1).unsqueeze(2)
       
        trg_mask = (trg != self.pad_idx).unsqueeze(1).unsqueeze(2)&torch.tril(torch.ones((trg_len,trg_len), device = self.device)).bool()
        
        #pos
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        #pos: [b, l]

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        # pos_emb = self.pos_emb(pos) #[b, l, h]
        # trg_emb = self.trg_emb(trg) #[b, l, h]
        
        # x = pos_emb + trg_emb * self.scale #[b, l, h]
        # x = self.dropout(x)
        
        for layer in self.layers:
            trg, attention = layer(trg, src, trg_mask, src_mask)
        
        output = self.fc(trg)
        
        return output, attention

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
INPUT_DIM = len(vocab)
OUTPUT_DIM = len(vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

# SRC_PAD_IDX = PAD_IDX
# TRG_PAD_IDX = PAD_IDX
model = dec.apply(initialize_weights).to(device)
model

Decoder(
  (pos_embedding): Embedding(100, 256)
  (tok_embedding): Embedding(385, 256)
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0): DecoderLayer(
      (norm_att_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (full_att_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (self_attention): MultiHeadAttentionLayer(
        (fc_q): Linear(in_features=256, out_features=256, bias=True)
        (fc_k): Linear(in_features=256, out_features=256, bias=True)
        (fc_v): Linear(in_features=256, out_features=256, bias=True)
        (fc): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): PositionwiseFeedforwardLayer(
        (fc1): Linear(in_features=256, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
 

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx) #combine softmax with cross entropy
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 1,804,417 trainable parameters


In [ ]:
def get_batch(data, seq_len, idx):
    #this data is from get_data()
    #train_data.shape # [batch_size, number of batches....]
    src    = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]  #target simply is ahead of src by 1            
    return src, target

In [ ]:
def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]


    
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        optimizer.zero_grad()
        # hidden = model.detach_hidden(hidden)

        src, target = get_batch(data, seq_len, idx) #src, target: [batch size, seq len]
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        # hidden = model.init_hidden(src)
        prediction, hidden = model(target,src)               

        prediction = prediction.reshape(batch_size * seq_len, -1)  #prediction: [batch size * seq len, vocab size]  
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        print(loss)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) #prevent gradient explosion - clip is basically 
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [ ]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    # hidden = model.init_hidden(batch_size, device)
    try:
      with torch.no_grad():
          for idx in range(0, num_batches - 1, seq_len):
              # hidden = model.detach_hidden(hidden)
              src, target = get_batch(data, seq_len, idx)
              src, target = src.to(device), target.to(device)
              batch_size= src.shape[0]

              #prediction, hidden = model(src, 0, target)
              prediction, hidden = model(target,src)
              prediction = prediction.reshape(batch_size * seq_len, -1)
              target = target.reshape(-1)
              itos = vocab.get_itos()
              # print(target.size(),prediction.size())
              # print(itos[torch.multinomial(torch.softmax(prediction[:, -1] / 1, dim=-1) , num_samples=1).item()  ])
              
              loss = criterion(prediction, target)
              epoch_loss += loss.item() * seq_len
          return epoch_loss / num_batches
    except:
      print(".")
    

In [ ]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, device, seed=3407):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    # hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)

            prediction, hidden = model(src,src)
            
            #prediction: [batch size, seq len, vocab size]
            #prediction[:, -1]: [batch size, vocab size] #probability of last vocab
            
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1)  
            prediction = torch.multinomial(probs, num_samples=1).item()    
            
            while prediction == vocab['<unk>']: #if it is unk, we sample again
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<eos>']:    #if it is eos, we stop
                break
#torch.multinomial(torch.softmax(prediction[:, -1] / temperature, dim=-1) , num_samples=1).item()
            indices.append(prediction) #autoregressive, thus output becomes input

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

In [ ]:
# n_epochs = 5
# seq_len  = 12
# clip    = 0.25

# lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

# best_valid_loss = float('inf')

# for epoch in range(n_epochs):
#     train_loss = train(model, train_data, optimizer, criterion, 
#                 batch_size, seq_len, clip, device)
#     valid_loss = evaluate(model, valid_data, criterion, batch_size, 
#                 seq_len, device)

#     lr_scheduler.step(valid_loss)

#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'predictor_weight.pt')
#     print(f'\tepoch: {epoch+1}')
#     print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
#     print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

ZeroDivisionError: ignored

In [ ]:
prompt = 'import pandas'
max_seq_len = 5
seed = 0
        #superdiverse   more diverse
temperatures = [0.5, 0.7, 0.75, 0.8, 1.0] 
#sample from this distribution higher probability will get more change
for temperature in temperatures:
    generation = generate(prompt, max_seq_len, temperature, model, tokenizer, 
                          vocab, device, seed)
    print(str(temperature)+'\n'+' '.join(generation)+'\n')

0.5
import pandas centrality want 32 python asin

0.7
import pandas centrality tsos 32 python score

0.75
import pandas centrality tsos 32 python score

0.8
import pandas centrality tsos 32 python score

1.0
import pandas centrality tsos 32 run_setting parameter

